In [231]:
import pygame
import numpy as np
import numpy.linalg as la
import random as rdm
import math as mh

FPS = 50
WIN_WIDTH = 1000
WIN_HEIGHT = 600
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (125, 125, 125)
LIGHT_BLUE = (64, 128, 255)
GREEN = (0, 128, 0)
RED = (255, 0, 0)
LIME = (0, 255, 0)
YELLOW = (225, 225, 0)
ORANGE = (255, 165, 0)
PINK = (230, 50, 230)
BGCOLOR = WHITE
ROBOTCOLOR = GRAY
WALLCOLOR = BLACK

class VisualObj:
    def draw(self):
        pass

class ActiveObj(VisualObj):
    def action(self):
        pass
    
class GeomObj(ActiveObj):
    def getDist(self, oPos):
        pass
    def getXDiff(self, oPos):
        pass
    def getYDiff(self, oPos):
        pass
    def getYDist(self, oPos):
        return abs(self.getYDiff(oPos))
    def isAbove(self, oPos):
        return self.getYDiff(oPos) > 0
    
class Robot(GeomObj):
    def __init__(self, pos, V = np.array([0, 0]), Rvis = 10, r = 3, color = ROBOTCOLOR):
        self._live = True
        self._pos = pos
        self._V = V
        self._r = r
        self._color = color
    def rise(self):
        self._live = True
    def kill(self):
        self._live = False
    def isLive(self):
        return self._live
    def teleportation(self, tPos):
        self._pos = tPos
    def getPos(self):
        return self._pos
    def getDist(self, oPos):
        return la.norm(self._pos - oPos)
    def getXDiff(self, oPos):
        return self._pos[0] - oPos[0]
    def getYDiff(self, oPos):
        return self._pos[1] - oPos[1]
    def setV(self, V):
        self._V = V
    def inN(self, RPos, RVis):
        return self.getDist(RPos) < RVis
    
    def draw(self, sc, O):
        DispPos = self._pos + O;
        pygame.draw.circle(sc, self._color, (int(DispPos[0]), int(DispPos[1])), self._r)
    def hide(self, sc, O):
        DispPos = self._pos + O;
        pygame.draw.circle(sc, BGCOLOR, (int(DispPos[0]), int(DispPos[1])), self._r)
    def action(self):
        self._pos = self._pos + self._V;
    
class Wall(GeomObj):
    def __init__(self, A, B, width = 3, color = WALLCOLOR):
        self._A = A
        self._B = B
        self._color = color
        self._width = width
    def isLive(self):
        return True
    def getDist(self, oPos):
        v = self._A - self._B
        v = v / la.norm(v)
        oPos = oPos - self._A
        return abs(v[1] * oPos[0] - v[0] * oPos[1])
    def getYDiff(self, oPos):
        v = self._A - self._B
        return self._A[1] + (oPos[0] - self._A[0]) / v[0] * v[1] - oPos[1]
    def inN(self, RPos, RVis):
        return False
    def draw(self, sc, O):
        pygame.draw.line(sc, self._color, tuple(self._A + O), tuple(self._B + O), self._width)

class CurvedWall(Wall):
    def __init__(self, func, width = 3, color = WALLCOLOR, drawStep = 3):
        self._f = func
        self._width = width
        self._color = color
        self._drawStep = drawStep
    def getDist(self, oPos):
        return self.getYDiff(oPos) #Вообще говоря расстояние до кривой нам не особо интересно
    def getYDiff(self, oPos):
        return self.getY(oPos[0]) - oPos[1]
    def getY(self, X):
        return self._f(X)
    def draw(self, sc, O):
        X = -O[0]
        Y = self._f(X)
        while X + O[0] < WIN_WIDTH:
            nX = X + self._drawStep
            nY = self._f(nX)
            pygame.draw.line(sc, self._color, (X + O[0], Y + O[1]), (nX + O[0], nY + O[1]), self._width)
            X = nX
            Y = nY
        

class Scene(ActiveObj):
    def __init__(self, sc, objs, GlobalCenter = np.array([0, 0])):
        self._sc = sc
        self._objects = objs
        self._O = GlobalCenter
    def draw(self):
        for obj in self._objects:
            if obj.isLive():
                obj.draw(self._sc, self._O)
    def action(self):
        for obj in self._objects:
            if obj.isLive():
                obj.action()  
            
def gamma(arg, param):
    return param * arg / (1 + arg * arg) ** 0.5

def theta(arg, param):
    return param * arg

def calcL(objs, RPos, RVis):
    res = RVis
    for obj in objs:
        if obj.isLive() and obj.getDist(RPos) < RVis:
            YDist = obj.getYDist(RPos)
            res = min(res, YDist)
    return res

def calcN(objs, RID):
    res = []
    for Id, obj in enumerate(objs):
        if obj.isLive() and Id != RID and obj.inN(objs[RID].getPos(), RVis):
            res.append(Id)
    return res

def getAbove(objs, RID):
    res = []
    for Id, obj in enumerate(objs):
        if obj.isLive() and Id != RID and obj.isAbove(objs[RID].getPos()):
            res.append(obj)
    return res

def getBelow(objs, RID):
    res = []
    for Id, obj in enumerate(objs):
        if obj.isLive() and Id != RID and not obj.isAbove(objs[RID].getPos()):
            res.append(obj)
    return res

def calcV(objs, N, LPlus, LMinus, V_, p1, p2, RID):
    Vx = 0
    for j in N:
        Vx = Vx + gamma(objs[j].getXDiff(objs[RID].getPos()), p1)
    if len(N) > 0:
        Vx = Vx / len(N) + V_
    else:
        Vx = V_
    Vy = theta(LPlus, p2) - theta(LMinus, p2)
    return np.array([Vx, Vy])

#Вернёт координаты случайно точки круга радиуса R с центров в (OX, OY) (равномерное распределение)
def getRndCrcPnt(R, OX = 0, OY = 0):
    X = rdm.uniform(-R, R)
    Y = rdm.uniform(-R, R)
    #Генерируем случайную точку внутри квадрата пока не попадём в круг
    while X ** 2 + Y ** 2 > R ** 2:
        X = rdm.uniform(-R, R)
        Y = rdm.uniform(-R, R)
        
    return np.array([X + OX, Y + OY])
      
#задаются константы
W = 250
rCnt = 15
minRCnt = 7
killRisePeriod = 800
maxKillGroupSize = 15
RVis = 70
maxV = 2
V_ = 0.8
CC = 0.1
Mult = 10
bias = 0
O = np.array([0, 300])

#проверка на допустимость констант
if W / minRCnt > RVis:
    print('Плохое соотношение W, minRCnt, RVis')
if W / rCnt > RVis:
    print('Плохое соотношение W, rCnt, RVis')
else:
    #вычисление коэффициентов для управляющего алгоритма
    BB = CC * theta(RVis, 1)
    p1 = (((1 + BB ** 2) * maxV ** 2 - (BB * V_) ** 2) ** 0.5 - V_) / (1 + BB ** 2)
    p2 = Mult * CC * p1
    D = (RVis ** 2 - (W / rCnt) ** 2) ** 0.5
    liveCnt = rCnt
    
    #случайно раскидываются роботы в полосе и инициализируются стенки
    #objs = [Robot(np.array([rdm.uniform(0, D), rdm.uniform(-W/2+50-bias, W/2-50-bias)])) for j in range(0, rCnt)]
    
    #случайно раскидываются роботы в окрестности центра коридора
    #objs = [Robot(getRndCrcPnt(D / 2, D / 2, -bias)) for j in range(0, rCnt)]
    
    #инициализируем роботов с нулевыми координатами
    objs = [Robot(np.array([0, 0])) for j in range(0, rCnt)]
    
    #Прямые стенки
    #objs.append(Wall(A = np.array([-10, -W / 2 - bias]), B = np.array([WIN_WIDTH + 10, -W / 2 + bias])))
    #objs.append(Wall(A = np.array([-10, W / 2 - bias]), B = np.array([WIN_WIDTH + 10, W / 2 + bias])))
    
    #Кривые параллельные стенки
        #Самое простое
    #objs.append(CurvedWall(lambda x: 40 * math.sin(x/20) - W/2))
    #objs.append(CurvedWall(lambda x: 40 * math.sin(x/20) + W/2))
    
        #Здесь иногда случаются проходы сквозь в стену но в целом всё хорошо
    #objs.append(CurvedWall(lambda x: 10 * (math.sin(x/5) + math.sin(x/7)) - W/2))
    #objs.append(CurvedWall(lambda x: 10 * (math.sin(x/5) + math.sin(x/7)) + W/2))
    
        #
    objs.append(CurvedWall(lambda x: 10*mh.sin(7**0.5*x/19) + 15*mh.sin(3**0.5*x/29) + 7*mh.sin(2**0.5*x/5) - W/2))
    objs.append(CurvedWall(lambda x: 10*mh.sin(7**0.5*x/19) + 15*mh.sin(3**0.5*x/29) + 7*mh.sin(2**0.5*x/5) + W/2))
    
        #Плохой тест
    #objs.append(CurvedWall(lambda x: 20 * (math.sin(x/5) + math.sin(x/7)) - W/2))
    #objs.append(CurvedWall(lambda x: 20 * (math.sin(x/5) + math.sin(x/7)) + W/2))
    
        #Кривые и смещенные
    #objs.append(CurvedWall(lambda x: 10 * math.sin(x/5) + x * 0.2 - W/2))
    #objs.append(CurvedWall(lambda x: 10 * math.sin(x/5) + x * 0.2 + W/2))
    
        #
    #objs.append(CurvedWall(lambda x: 10 * (math.sin(x/7) + math.sin(x/13)) + x * 0.25 - W))
    #objs.append(CurvedWall(lambda x: 10 * (math.sin(x/7) + math.sin(x/13)) + x * 0.25))
    
    #Кривые сужающиеся расширяющиеся стенки
        #Самое простое
    #objs.append(CurvedWall(lambda x:  40 * math.sin(x/20) - W/2))
    #objs.append(CurvedWall(lambda x: -40 * math.sin(x/20) + W/2))
    
        #
    #objs.append(CurvedWall(lambda x:  10 * (math.sin(x/5) + math.sin(x/7)) - W/2))
    #objs.append(CurvedWall(lambda x: -10 * (math.sin(x/5) + math.sin(x/7)) + W/2))
    
        #Плохой тест
    #objs.append(CurvedWall(lambda x:  20 * (math.sin(x/5) + math.sin(x/7)) - W/2))
    #objs.append(CurvedWall(lambda x: -20 * (math.sin(x/5) + math.sin(x/7)) + W/2))
    
        #Кривые и смещённые
    #objs.append(CurvedWall(lambda x:  10 * math.sin(x/5) + x * 0.2 - W/2))
    #objs.append(CurvedWall(lambda x: -10 * math.sin(x/5) + x * 0.2 + W/2))
    
        #
    #objs.append(CurvedWall(lambda x:  10 * (math.sin(x/7) + math.sin(x/13)) + x * 0.25 - W))
    #objs.append(CurvedWall(lambda x: -10 * (math.sin(x/7) + math.sin(x/13)) + x * 0.25))
    
    #Раскидываем роботов в начале коридора в окрестности его центра
    for j in range(0, rCnt):
        objs[j].teleportation(getRndCrcPnt(D/2, D/2, (objs[rCnt].getY(D/2) + objs[rCnt+1].getY(D/2))/2))
    
    #инициализация симуляции
    pygame.init()

    clock = pygame.time.Clock()

    sc = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))

    sm = Scene(sc, objs, O)

    sc.fill(BGCOLOR)
    #рисуем стенки
    objs[rCnt].draw(sc, O)
    objs[rCnt + 1].draw(sc, O)
    #симуляция
    running = 1;
    while running >= 0:
        #sc.fill(BGCOLOR)

        for i in pygame.event.get():
            if i.type == pygame.QUIT: running = -100500;

        #sm.draw()
        #рисуются живые роботы
        for j in range(0, rCnt):
            if objs[j].isLive():
                objs[j].draw(sc, O)

        pygame.display.update()
        
        #все перемещения происходят здесь
        numDisappear = 0
        for j in range(0, rCnt):
            if objs[j].isLive():
                LMinus = calcL(getBelow(objs, j), objs[j].getPos(), RVis)
                LPlus = calcL(getAbove(objs, j), objs[j].getPos(), RVis)
                N = calcN(objs, j)
                newV = calcV(objs, N, LPlus, LMinus, V_, p1, p2, j)
                absV = la.norm(newV)
                if absV > maxV:
                    newV = newV / absV * maxV
                objs[j].setV(newV)
                if (objs[j].getPos() + O)[0] > WIN_WIDTH + 10:
                    numDisappear = numDisappear + 1
                 
        if running % killRisePeriod == 0:
            #убиваем или воскрешаем роботов
            doKill = (np.random.randint(2) == 1 or liveCnt == rCnt) and (liveCnt > minRCnt)
            if doKill:
                #убиваем роботов
                killGroupSize = np.random.randint(min(liveCnt - minRCnt + 1, maxKillGroupSize + 1))
                print('Killed: ' + str(killGroupSize))
                for j in range(0, killGroupSize):
                    Id = np.random.randint(rCnt)
                    while not objs[Id].isLive():
                        Id = np.random.randint(rCnt)
                    liveCnt = liveCnt - 1
                    objs[Id].kill()
            else:
                #воскрешаем роботов
                risePnt = np.array([0, 0])
                for j in range(0, rCnt):
                    if objs[j].isLive():
                        risePnt = risePnt + objs[j].getPos()
                risePnt = risePnt / liveCnt
                riseCnt = 0
                for j in range(0, rCnt):
                    if not objs[j].isLive():
                        doRise = np.random.randint(2) == 0 #вероятность воскресить 1/2
                        if doRise:
                            #Воскрешение в пределах полосы
                            #X = rdm.uniform(risePnt[0] - D/2, risePnt[0] + D/2)
                            #Y = rdm.uniform(risePnt[1] - W/2 + 50, risePnt[1] + W/2 - 50)
                            #objs[j].teleportation(np.array([X, Y]))
                            
                            #Воскрешение в окресности центра строя
                            objs[j].teleportation(getRndCrcPnt(D/2, risePnt[0], risePnt[1]))
                            objs[j].rise()
                            riseCnt = riseCnt + 1
                liveCnt = liveCnt + riseCnt
                print('Risen:  ' + str(riseCnt)) 
        
        #для цикличности "телепортируем" роботов в начало коридора после того как все они заедут за границу
        #наблюдаемой зоны
        #Просто обнуляем координаты по X (ну ещё и смещение учитывается, но не особо что-то даёт в общем случае)
        #if numDisappear >= liveCnt:
        #    for j in range(0, rCnt):
        #        if objs[j].isLive():
        #           objs[j].teleportation(objs[j].getPos() - np.array([WIN_WIDTH, 2 * bias]))
        
        #Смотрим на расстояние до стеночки каждого робота и телепортируем так, чтобы оно сохранялось
        if numDisappear >= liveCnt:
            for j in range(0, rCnt):
                if objs[j].isLive():
                    tpX = objs[j].getPos()[0] - WIN_WIDTH
                    tpY = objs[rCnt].getY(tpX) - objs[rCnt].getYDiff(objs[j].getPos())
                    objs[j].teleportation(np.array([tpX, tpY]))
                    
        #стираем роботов
        for j in range(0, rCnt):
            objs[j].hide(sc, O)
        
        sm.action()
        
        running = running + 1
        clock.tick(FPS)
    pygame.quit()

In [111]:
class Worker:
    def __init__(self, DoFunc):
        self._DoFunc = DoFunc
    def Do(self, arg):
        self._DoFunc(arg)
        
def DoF(arg):
    print(arg)
    
def DoF2(arg):
    print('Kek')
    
worker = Worker(DoF)
worker.Do('Hello!!!')
worker2 = Worker(DoF2)
worker2.Do('Buba')

Hello!!!
Kek


In [126]:
import math
math.sin(3)

0.1411200080598672